In [1]:
import os
PROJ_ROOT = os.path.abspath(os.path.join(os.pardir))
print(PROJ_ROOT)

/home/bruno/Desktop/coarse-discourse-validation


## Annotations download from paper's repo

In [16]:
import requests

url = 'https://raw.githubusercontent.com/google-research-datasets/coarse-discourse/master/coarse_discourse_dataset.json'
filename = os.path.join(PROJ_ROOT,'data', 'external', 'coarse_discourse_dataset.json')

with open(filename, 'wb') as jsonfile:
    response = requests.get(url)
    jsonfile.write(response.content)

## Get reddit data from API

In [31]:
import json
import praw

jsonfile = open(filename, 'rb')

try:
    api_id = os.environ['REDDIT_CLIENT_ID']
except KeyError:
    print("Please set your client id from Reddit API")

try:
    api_secret = os.environ['REDDIT_CLIENT_SECRET']
except KeyError:
    print("Please set your client secret from Reddit API")

In [32]:
reddit = praw.Reddit(client_id=api_id,
                     client_secret=api_secret,
                     user_agent='linux:coarse-discourse-validation:0.1 (by /u/brunompac)')

lines = jsonfile.readlines()
thread = json.loads(lines[13]) # Random line for simulation

submission = reddit.submission(url=thread['url']) # Gets thread data
submission.comment_sort = 'best'
submission.comment_limit = 40

post = thread['posts'][-1] # Random thread comment

In [33]:
for p in thread['posts']:
    print(p['id'])

t3_2kmipp
t1_clmr1lk
t1_clmr43x
t1_clmp7qq
t1_clmp08s
t1_cln2wr5
t1_cln8eha
t1_clnylli
t1_clmpxa7
t1_clmq0s6
t1_clmqcfv


In [6]:
print(thread['title'])
print(thread['url'])
print('\t'+submission.selftext)

Looking for Oracle of Seasons, Trading Club Nintendo games.
https://www.reddit.com/r/3DS/comments/1riref/looking_for_oracle_of_seasons_trading_club/
	I have 350 coins, and don't really want anything on there atm.  Anyone willing to trade an OoS code for any of the games or stuff on offer?


In [7]:
full_submission_id = 't3_'+submission.id

print(full_submission_id)
print(post['id'])

t3_1riref
t1_cdo8mp8


For explanation on id's and the `'t3_'`, check **type prefixes** at https://www.reddit.com/dev/api/

In [8]:
submission.comments.replace_more(limit=0)

# Gets comment from submission that has the same id as `post`
post_short_id = post['id'][3:] # Removes type prefix
comment = list(filter(lambda x: x.id == post_short_id, submission.comments.list()))[0]

post['body'] = comment.body
if comment.author:
    post['author'] = comment.author.name
    
print(post['body'])
try:
    print('\t- '+post['author'])
except:
    pass

I would consider other 3ds code trades as well.
	- tamag21


## Reddit data download from dump

<span style="color:red">****</span>

#### Test the hypothesis that the first post's id is always the same as the thread's id

This will be important as the bigquery table doesn't have url field.

WILL TAKE ~5 HOURS!

In [9]:
import time

for l in lines:
    t = json.loads(l)
    
    s = reddit.submission(url=t['url'])
    
    if (t['posts'][0]['id'][3:] != s.id):
        print("Unconsistency found!\nsubmission: {} - {}".format(s.id, s.title))
        print("thread: {} - {}".format(t['posts'][0]['id'], t['url']))
    
    # print("thread \"{}\" done!".format(t['title']))
    time.sleep(2)

KeyboardInterrupt: 

#### Query to get the top 40 comments from thread

Where `link_id` is the thread's id.

Check the calculation at http://www.evanmiller.org/how-not-to-sort-by-average-rating.html.

In [39]:
query = """
SELECT *, ((ups + 1.9208) / (ups + downs) - 
                   1.96 * SQRT((ups * downs) / (ups + downs) + .9604) / 
                          (ups + downs)) / (1 + 3.8416 / (ups + downs)) 
       AS ci_lower_bound FROM [fh-bigquery:reddit_comments.all] WHERE ((link_id='t3_2kmipp')) 
       ORDER BY ci_lower_bound DESC LIMIT 40
"""